**Importing neccessary features that would be needed during the execution of the program**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Loading the data sets that will be used in training and testing i.e player_21 and player_22**

In [ ]:

df_train = pd.read_csv('/content/drive/My Drive/AI Project/players_21.csv')
df_test = pd.read_csv('/content/drive/My Drive/AI Project/players_22.csv')

<ipython-input-2-1ecbbc1ea746>:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('/content/drive/My Drive/AI Project/players_22.csv')


**Dropping uneccessary columns in the dataframes**

In [ ]:
columns_to_drop = ['player_url', 'player_positions', 'player_face_url', 'club_logo_url', 'club_flag_url', 'nation_logo_url', 'nation_flag_url']
df_train = df_train.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)

Inspecting the initial rows of our DataFrame. This allows us to see the first few rows of our dataset, helping us understand the structure of the data and the types of values present in each column. It gives us a sense of what the data looks like and will help us decide on further analysis and preprocessing steps.

In [ ]:
df_train

,sofifa_id,short_name,long_name,overall,potential,value_eur,wage_eur,age,dob,height_cm,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,158023,L. Messi,Lionel Andrés Messi Cuccittini,93,93,103500000.0,560000.0,33,1987-06-24,170,...,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3,19+3
1,20801,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,92,92,63000000.0,220000.0,35,1985-02-05,187,...,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3,20+3
2,188545,R. Lewandowski,Robert Lewandowski,91,91,111000000.0,240000.0,31,1988-08-21,184,...,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3
3,190871,Neymar Jr,Neymar da Silva Santos Júnior,91,91,132000000.0,270000.0,28,1992-02-05,175,...,62+3,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3,20+3
4,192985,K. De Bruyne,Kevin De Bruyne,91,91,129000000.0,370000.0,29,1991-06-28,181,...,80+3,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,257710,Zhang Mengxuan,张梦炫,47,52,70000.0,1000.0,21,1999-04-26,177,...,41+2,41+2,41+2,42+2,45+2,47+2,47+2,47+2,45+2,15+2
18940,257933,Huang Wenzhou,黄文卓,47,53,70000.0,1000.0,21,1999-01-07,174,...,48+2,48+2,48+2,47+2,47+2,46+2,46+2,46+2,47+2,15+2
18941,257936,Song Yue,宋岳,47,47,45000.0,2000.0,28,1991-11-20,185,...,47,47,47,47,47,46+1,46+1,46+1,47,11+2
18942,258736,V. Da Silva,Ivanilson Loforte Tique Da Silva,47,67,130000.0,500.0,17,2003-03-30,171,...,32+2,32+2,32+2,35+2,33+2,26+2,26+2,26+2,33+2,14+2


In [ ]:
df_test

,sofifa_id,short_name,long_name,overall,potential,value_eur,wage_eur,age,dob,height_cm,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,158023,L. Messi,Lionel Andrés Messi Cuccittini,93,93,78000000.0,320000.0,34,1987-06-24,170,...,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3
1,188545,R. Lewandowski,Robert Lewandowski,92,92,119500000.0,270000.0,32,1988-08-21,185,...,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3
2,20801,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,91,91,45000000.0,270000.0,36,1985-02-05,187,...,59+3,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3,20+3
3,190871,Neymar Jr,Neymar da Silva Santos Júnior,91,91,129000000.0,270000.0,29,1992-02-05,175,...,63+3,63+3,63+3,67+3,62+3,50+3,50+3,50+3,62+3,20+3
4,192985,K. De Bruyne,Kevin De Bruyne,91,91,125500000.0,350000.0,30,1991-06-28,181,...,80+3,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962,Song Defu,宋德福,47,52,70000.0,1000.0,22,1999-01-02,180,...,46+2,46+2,46+2,48+2,48+2,46+2,46+2,46+2,48+2,15+2
19235,262040,C. Porter,Caoimhin Porter,47,59,110000.0,500.0,19,2002-03-18,175,...,46+2,46+2,46+2,49+2,48+2,44+2,44+2,44+2,48+2,14+2
19236,262760,N. Logue,Nathan Logue-Cunningham,47,55,100000.0,500.0,21,2000-02-07,178,...,46+2,46+2,46+2,47+2,47+2,45+2,45+2,45+2,47+2,12+2
19237,262820,L. Rudden,Luke Rudden,47,60,110000.0,500.0,19,2002-04-23,173,...,29+2,29+2,29+2,33+2,32+2,26+2,26+2,26+2,32+2,15+2


Dropping columns in our dataset that have more than 30% missing values

In [ ]:
# Drop columns with more than 30% missing values
missing_percentage_train = df_train.isnull().sum() / len(df_train) * 100
columns_to_drop_train = missing_percentage_train[missing_percentage_train > 30].index
df_train.drop(columns_to_drop_train, axis=1, inplace=True)

These are the columns that have been dropped in the datasets

In [ ]:
columns_to_drop_train

Index(['club_loaned_from', 'nation_team_id', 'nation_position',
       'nation_jersey_number', 'player_tags', 'player_traits',
       'goalkeeping_speed'],
      dtype='object')

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 96 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    18944 non-null  int64  
 1   short_name                   18944 non-null  object 
 2   long_name                    18944 non-null  object 
 3   overall                      18944 non-null  int64  
 4   potential                    18944 non-null  int64  
 5   value_eur                    18707 non-null  float64
 6   wage_eur                     18719 non-null  float64
 7   age                          18944 non-null  int64  
 8   dob                          18944 non-null  object 
 9   height_cm                    18944 non-null  int64  
 10  weight_kg                    18944 non-null  int64  
 11  club_team_id                 18719 non-null  float64
 12  club_name                    18719 non-null  object 
 13  league_name     

Dropping columns with more than 30% missing values in player_22

---



In [ ]:
missing_percentage_test = df_test.isnull().sum() / len(df_test) * 100
columns_to_drop_test = missing_percentage_test[missing_percentage_test > 30].index
df_test.drop(columns_to_drop_test, axis=1, inplace=True)

In [ ]:
columns_to_drop_test

Index(['club_loaned_from', 'nation_team_id', 'nation_position',
       'nation_jersey_number', 'player_tags', 'player_traits',
       'goalkeeping_speed'],
      dtype='object')

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 96 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    19239 non-null  int64  
 1   short_name                   19239 non-null  object 
 2   long_name                    19239 non-null  object 
 3   overall                      19239 non-null  int64  
 4   potential                    19239 non-null  int64  
 5   value_eur                    19165 non-null  float64
 6   wage_eur                     19178 non-null  float64
 7   age                          19239 non-null  int64  
 8   dob                          19239 non-null  object 
 9   height_cm                    19239 non-null  int64  
 10  weight_kg                    19239 non-null  int64  
 11  club_team_id                 19178 non-null  float64
 12  club_name                    19178 non-null  object 
 13  league_name     

Separating data into numeric and non_numeric features in both datasets

In [ ]:
numeric_features_train = df_train.select_dtypes(include=['number'])
non_numeric_features_train = df_train.select_dtypes(exclude=['number'])

numeric_features_test = df_test.select_dtypes(include=['number'])
non_numeric_features_test = df_test.select_dtypes(exclude=['number'])



In [ ]:
numeric_features_train.info() , non_numeric_features_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 57 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    18944 non-null  int64  
 1   overall                      18944 non-null  int64  
 2   potential                    18944 non-null  int64  
 3   value_eur                    18707 non-null  float64
 4   wage_eur                     18719 non-null  float64
 5   age                          18944 non-null  int64  
 6   height_cm                    18944 non-null  int64  
 7   weight_kg                    18944 non-null  int64  
 8   club_team_id                 18719 non-null  float64
 9   league_level                 18719 non-null  float64
 10  club_jersey_number           18719 non-null  float64
 11  club_contract_valid_until    18719 non-null  float64
 12  nationality_id               18944 non-null  int64  
 13  weak_foot       

(None, None)

In [ ]:
numeric_features_test.info() , non_numeric_features_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 57 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    19239 non-null  int64  
 1   overall                      19239 non-null  int64  
 2   potential                    19239 non-null  int64  
 3   value_eur                    19165 non-null  float64
 4   wage_eur                     19178 non-null  float64
 5   age                          19239 non-null  int64  
 6   height_cm                    19239 non-null  int64  
 7   weight_kg                    19239 non-null  int64  
 8   club_team_id                 19178 non-null  float64
 9   league_level                 19178 non-null  float64
 10  club_jersey_number           19178 non-null  float64
 11  club_contract_valid_until    19178 non-null  float64
 12  nationality_id               19239 non-null  int64  
 13  weak_foot       

(None, None)

Imputing missing values in the numeric and non numeric columns for both data sets

In [ ]:
numeric_imputer = SimpleImputer(strategy='mean')
df_train[numeric_features_train.columns] = numeric_imputer.fit_transform(df_train[numeric_features_train.columns])
df_test[numeric_features_test.columns] = numeric_imputer.transform(df_test[numeric_features_test.columns])

non_numeric_imputer = SimpleImputer(strategy='most_frequent')
df_train[non_numeric_features_train.columns] = non_numeric_imputer.fit_transform(df_train[non_numeric_features_train.columns])
df_test[non_numeric_features_test.columns] = non_numeric_imputer.transform(df_test[non_numeric_features_test.columns])

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 96 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    18944 non-null  float64
 1   short_name                   18944 non-null  object 
 2   long_name                    18944 non-null  object 
 3   overall                      18944 non-null  float64
 4   potential                    18944 non-null  float64
 5   value_eur                    18944 non-null  float64
 6   wage_eur                     18944 non-null  float64
 7   age                          18944 non-null  float64
 8   dob                          18944 non-null  object 
 9   height_cm                    18944 non-null  float64
 10  weight_kg                    18944 non-null  float64
 11  club_team_id                 18944 non-null  float64
 12  club_name                    18944 non-null  object 
 13  league_name     

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 96 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    19239 non-null  float64
 1   short_name                   19239 non-null  object 
 2   long_name                    19239 non-null  object 
 3   overall                      19239 non-null  float64
 4   potential                    19239 non-null  float64
 5   value_eur                    19239 non-null  float64
 6   wage_eur                     19239 non-null  float64
 7   age                          19239 non-null  float64
 8   dob                          19239 non-null  object 
 9   height_cm                    19239 non-null  float64
 10  weight_kg                    19239 non-null  float64
 11  club_team_id                 19239 non-null  float64
 12  club_name                    19239 non-null  object 
 13  league_name     

Encoding the categorical values in the non numeric columns using label encoder



In [ ]:
label_encoder = LabelEncoder()
for column in non_numeric_features_train.columns:
    df_train[column] = label_encoder.fit_transform(df_train[column])
for column in non_numeric_features_test.columns:
    df_test[column] = label_encoder.fit_transform(df_test[column])


Displaying first 5 rows of the encoded data

In [ ]:
df_train.head()

,sofifa_id,short_name,long_name,overall,potential,value_eur,wage_eur,age,dob,height_cm,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,158023.0,10059,10302,93.0,93.0,103500000.0,560000.0,33.0,942,170.0,...,136,136,136,133,110,59,59,59,110,18
1,20801.0,3261,3332,92.0,92.0,63000000.0,220000.0,35.0,391,187.0,...,110,110,110,126,104,66,66,66,104,20
2,188545.0,14329,14687,91.0,91.0,111000000.0,240000.0,31.0,1271,184.0,...,136,136,136,119,104,96,96,96,104,18
3,190871.0,13007,12964,91.0,91.0,132000000.0,270000.0,28.0,2438,175.0,...,116,116,116,139,110,52,52,52,110,20
4,192985.0,8987,9626,91.0,91.0,129000000.0,370000.0,29.0,2229,181.0,...,232,232,232,204,185,158,158,158,185,22


In [ ]:
df_test.head()

,sofifa_id,short_name,long_name,overall,potential,value_eur,wage_eur,age,dob,height_cm,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,158023.0,10117,10421,93.0,93.0,78000000.0,320000.0,34.0,719,170.0,...,119,119,119,117,99,56,56,56,99,18
1,188545.0,14468,14882,92.0,92.0,119500000.0,270000.0,32.0,1016,185.0,...,131,131,131,107,99,105,105,105,99,18
2,20801.0,3288,3379,91.0,91.0,45000000.0,270000.0,36.0,266,187.0,...,88,88,88,102,93,68,68,68,93,20
3,190871.0,13097,13094,91.0,91.0,129000000.0,270000.0,29.0,2143,175.0,...,113,113,113,123,104,56,56,56,104,20
4,192985.0,9005,9692,91.0,91.0,125500000.0,350000.0,30.0,1940,181.0,...,217,217,217,181,168,164,164,164,168,22


**PLAYER 21 FEATURE SELECTION AND SCALING**

Separating features and target variable for training the data from the player 21 data set.

In [ ]:
X_train = df_train.drop('overall', axis=1)
y_train = df_train['overall']

training a random forest regression model. It creates an instance of the RandomForestRegressor class and then fits the model to the training data (X_train and y_train).

In [ ]:
rf_model_train = RandomForestRegressor()
rf_model_train.fit(X_train, y_train)

RandomForestRegressor()

The above code is calculating the feature importances of a random forest model (`rf_model_train`) trained on a training dataset (`X_train`). It then creates a DataFrame (`feature_importance_df_train`) to store the feature importances, with columns for the feature names and their corresponding importances. Finally, it sorts the features in descending order based on their importance values.

In [ ]:
feature_importances_train = rf_model_train.feature_importances_
feature_importance_df_train = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances_train})
feature_importance_df_train = feature_importance_df_train.sort_values(by='Importance', ascending=False)

selecting the top 7 features based on their importance scores and then creating a new dataframe called `top_features_train_importances` which contains the top 7 features along with their importance scores.

In [ ]:
top_features_train = feature_importance_df_train.head(7)['Feature'].tolist()
top_features_train_importances = feature_importance_df_train.head(7)


Displaying the top 7 features

In [ ]:
top_features_train

['value_eur',
 'release_clause_eur',
 'dob',
 'potential',
 'movement_reactions',
 'age',
 'gk']

Displaying top 7 features with their importance score

In [ ]:
top_features_train_importances

,Feature,Importance
4,value_eur,0.676800
27,release_clause_eur,0.138859
7,dob,0.109620
3,potential,0.048887
47,movement_reactions,0.016394
6,age,0.002207
94,gk,0.000752


Importing the StandardScaler class from the sklearn.preprocessing module.Then separating the selected features and the target variable from the training data. The selected features are stored in the variable X_train, and the target variable is stored in the variable y_train.

In [ ]:
from sklearn.preprocessing import StandardScaler

X_train = df_train[top_features_train]
y_train = df_train['overall']

performing feature scaling on the training data using the StandardScaler() function from the scikit-learn library
Then displaying the datafram with the scaled data(features)


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

pd.DataFrame(X_train, columns=top_features_train)

,value_eur,release_clause_eur,dob,potential,movement_reactions,age,gk
0,13.073165,13.695696,-1.582441,3.586563,3.554438,1.655055,0.084727
1,7.809992,7.312715,-1.933771,3.422893,3.664174,2.080838,0.206313
2,14.047827,13.042079,-1.372662,3.259222,3.444701,1.229273,0.084727
3,16.776879,16.565484,-0.628555,3.259222,3.225227,0.590598,0.206313
4,16.387015,16.003782,-0.761819,3.259222,3.225227,0.803490,0.327900
...,...,...,...,...,...,...,...
18939,-0.368070,-0.432955,0.993560,-3.123941,-1.493455,-0.899642,-0.462413
18940,-0.368070,-0.431423,0.925334,-2.960270,-1.273981,-0.899642,-0.462413
18941,-0.371319,-0.433977,-0.674464,-3.942295,-1.932402,0.590598,-0.948759
18942,-0.360272,-0.421926,1.754245,-0.668878,-0.944771,-1.751207,-0.583999


**PLAYER 22 FEATURE SELECTION AND SCALING**




Separating features and target variable for training data (player_21)

In [ ]:
X_test_player22 = df_test.drop('overall', axis=1)
y_test_player22 = df_test['overall']


creating a new variable called "X_test_player22" and assigning it the value of a
subset of the "df_test" dataframe. The subset is created by selecting only the columns specified in the "top_features_train" variable.

In [ ]:
X_test_player22 = df_test[top_features_train]

X_test_player22

,value_eur,release_clause_eur,dob,potential,movement_reactions,age,gk
0,78000000.0,144300000.0,719,93.0,94.0,34.0,18
1,119500000.0,197200000.0,1016,92.0,93.0,32.0,18
2,45000000.0,83300000.0,266,91.0,94.0,36.0,20
3,129000000.0,238700000.0,2143,91.0,89.0,29.0,20
4,125500000.0,232200000.0,1940,91.0,91.0,30.0,22
...,...,...,...,...,...,...,...
19234,70000.0,114000.0,4570,52.0,53.0,22.0,9
19235,110000.0,193000.0,5699,59.0,49.0,19.0,7
19236,100000.0,175000.0,4958,55.0,46.0,21.0,3
19237,110000.0,239000.0,5735,60.0,48.0,19.0,9


Here we are scaling the features of Player 22's test data using a scaler object called "scaler". It then displays the scaled features in a pandas DataFrame with column names specified by the "top_features_train" variable.

In [ ]:
X_test_player22_scaled = scaler.transform(X_test_player22)
pd.DataFrame(X_test_player22_scaled, columns=top_features_train)

,value_eur,release_clause_eur,dob,potential,movement_reactions,age,gk
0,9.759315,14.298250,-1.724631,3.586563,3.554438,1.867947,0.084727
1,15.152443,19.700805,-1.535257,3.422893,3.444701,1.442164,0.084727
2,5.470804,8.068460,-2.013474,3.259222,3.554438,2.293730,0.206313
3,16.387015,23.939104,-0.816654,3.259222,3.005754,0.803490,0.206313
4,15.932173,23.275274,-0.946092,3.259222,3.225227,1.016381,0.327900
...,...,...,...,...,...,...,...
19234,-0.368070,-0.427134,0.730859,-3.123941,-0.944771,-0.686750,-0.462413
19235,-0.362872,-0.419066,1.450736,-1.978245,-1.383718,-1.325424,-0.583999
19236,-0.364171,-0.420904,0.978257,-2.632929,-1.712929,-0.899642,-0.827172
19237,-0.362872,-0.414368,1.473690,-1.814574,-1.493455,-1.325424,-0.462413


 Here we are splitting the dataset into training and testing sets. It uses the `train_test_split` function from the `sklearn.model_selection` module to randomly split the `X_train` and `y_train` datasets into `Xtrain`, `Xtest`, `ytrain`, and `ytest` sets. The `test_size` parameter is set to 0.2, which means that 20% of the data will be used for testing and 80% will be used for training.

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


The function evaluates a machine learning model by calculating the mean absolute error (MAE) and R-squared (R2) score between the predicted and true values.[param model: The model is the machine learning model that has been trained and is being evaluated, param X: The input features or independent variables used to make predictions , param y_true: The true values of the target variable (the actual values you are trying to predict),
 return: the mean absolute error (MAE) and the R-squared (R2) score.]

In [ ]:
def evaluate_model(model, X, y_true):
    y_pred = model.predict(X)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return  mae, r2

performing hyperparameter tuning and evaluation of a RandomForestRegressor modele using cross-validation.

In [ ]:

rf_model = RandomForestRegressor(random_state=42, n_jobs=-1)
rf_params = {'n_estimators': [100, 200,400], 'max_depth': [10,30,40]}
rf_grid = GridSearchCV(rf_model, rf_params, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
rf_grid.fit(Xtrain, ytrain)
rf_model = rf_grid.best_estimator_
cross_val_scores = cross_val_score(rf_model, Xtrain, ytrain, cv=5, scoring='neg_mean_absolute_error')
print("Cross-Validation MAE Scores:", -cross_val_scores)
rf_metrics = evaluate_model(rf_model, Xtest, ytest)
print('\nBest RandomForestRegressor Metrics (on Test Set):', rf_metrics)
print('\n')

Cross-Validation MAE Scores: [0.25067469 0.2482745  0.23937397 0.24375289 0.25200016]

Best RandomForestRegressor Metrics (on Test Set): (0.2151233834784901, 0.9959828666913827)




performing hyperparameter tuning and evaluation of an XGBoost regression model

In [ ]:

xgb_model = XGBRegressor(random_state=42, n_jobs=-1)
xgb_params = {'n_estimators': [50, 100, 200], 'max_depth': [2, 10, 20], 'learning_rate': [0.1, 0.001,0.01]}
xgb_grid = GridSearchCV(xgb_model, xgb_params, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
xgb_grid.fit(Xtrain, ytrain)
xgb_model = xgb_grid.best_estimator_
cross_val_scores_xgb = cross_val_score(xgb_model, Xtrain, ytrain, cv=5, scoring='neg_mean_absolute_error')
print("Cross-Validation MAE Scores for XGBoost:", -cross_val_scores_xgb)
xgb_metrics = evaluate_model(xgb_model, Xtest, ytest)
print('\nBest XGBoost Metrics (on Test Set):', xgb_metrics)
print('\n')

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Cross-Validation MAE Scores for XGBoost: [0.26042823 0.2578283  0.24570859 0.25870066 0.26085298]

Best XGBoost Metrics (on Test Set): (0.22477479734846142, 0.9958338998554809)




performing hyperparameter tuning for a Gradient Boosting Regressor model using GridSearchCV. It creates a GradientBoostingRegressor model with a random state of 42. It then defines a dictionary of hyperparameters to search over, including the number of estimators, maximum depth, and learning rate.

In [ ]:

gb_model = GradientBoostingRegressor(random_state=42)
gb_params = {'n_estimators': [50, 100, 200], 'max_depth': [2, 10, 20],'learning_rate': [0.1, 0.01,0.01]}
gb_grid = GridSearchCV(gb_model, gb_params, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
gb_grid.fit(Xtrain, ytrain)
gb_model = gb_grid.best_estimator_
cross_val_scores_gb = cross_val_score(gb_model, Xtrain, ytrain, cv=5, scoring='neg_mean_absolute_error')
print("Cross-Validation MAE Scores for Gradient Boosting:", -cross_val_scores_gb)
gb_metrics = evaluate_model(gb_model, Xtest, ytest)
print('\nBest Gradient Boosting Metrics (on Test Set):', gb_metrics)
print('\n')

The ensemble model combines the predictions of three individual models: rf_model, xgb_model, and gb_model. The ensemble model is trained on the Xtrain and ytrain data. After training, the model is evaluated using the evaluate_model function on the Xtest and ytest data. The evaluation metrics are then printed.

In [37]:
from sklearn.model_selection import GridSearchCV

ensemble_model = VotingRegressor(estimators=[('rf', rf_model), ('xgb', xgb_model), ('gb', gb_model)], n_jobs=-1)
ensemble_params = {
    'weights': [[1, 1, 1], [2, 1, 1], [1, 2, 1], [1, 1, 2], [2, 2, 1], [2, 1, 2], [1, 2, 2]]
    }
grid_search_ensemble = GridSearchCV(ensemble_model, ensemble_params, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search_ensemble.fit(Xtrain, ytrain)
print("Best Parameters for VotingRegressor:", grid_search_ensemble.best_params_)
print("Best Cross-Validation MAE Score for VotingRegressor:", -grid_search_ensemble.best_score_)
best_ensemble_model = grid_search_ensemble.best_estimator_
ensemble_metrics_test = evaluate_model(best_ensemble_model, Xtest, ytest)
print('VotingRegressor Test Metrics:', ensemble_metrics_test)



/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Parameters for VotingRegressor: {'weights': [1, 1, 1]}
Best Cross-Validation MAE Score for VotingRegressor: 0.23949566504870642
VotingRegressor Test Metrics: (0.20983791482637595, 0.9964761181529527)


calls the `evaluate_model` function, passing in the `rf_model` (RandomForestRegressor model), `X_test_player22_scaled` (scaled test dataset for Player 22), and `y_test_player22` (target variable for Player 22). The function returns the metrics for the model's performance, which are then printed out.

In [38]:

rf_player22_metrics = evaluate_model(rf_model, X_test_player22_scaled, y_test_player22)
print('RandomForestRegressor Metrics on Player 22:', rf_player22_metrics)
print('\n')

RandomForestRegressor Metrics on Player 22: (0.41478468215603703, 0.9888725087608506)




Calling the function `evaluate_model` with the XGBoost model, the scaled test dataset (`X_test_player22_scaled`), and the corresponding labels(`y_test_player22`).

In [39]:

xgb_player22_metrics = evaluate_model(xgb_model, X_test_player22_scaled, y_test_player22)
print('XGBoost Metrics on Player 22:', xgb_player22_metrics)
print('\n')

XGBoost Metrics on Player 22: (0.44372538805466455, 0.9872474358809686)




The code is evaluating the performance of the GradientBoostingRegressor model on the Player 22 data. It uses the `evaluate_model` function to calculate the mean absolute error (MAE) and R-squared (R2) scores between the predicted values (`gb_model.predict(X_test_player22_scaled)`) and the actual values (`y_test_player22`). The calculated metrics are then stored in the `gb_player22_metrics` variable and printed out.

In [40]:
gb_player22_metrics = evaluate_model(gb_model, X_test_player22_scaled, y_test_player22)
print('GradientBoostingRegressor Metrics on Player 22:', gb_player22_metrics)
print('\n')

GradientBoostingRegressor Metrics on Player 22: (0.47413183465303144, 0.9847915517766906)




calling the function `evaluate_model` with the ensemble model, the scaled test data (`X_test_player22_scaled`), and the corresponding target variable (`y_test_player22`). The function returns the metrics of the model's performance, which are then printed out.


In [41]:
ensemble_player22_metrics = evaluate_model(best_ensemble_model, X_test_player22_scaled, y_test_player22)
print('VotingRegressor Metrics on Player 22:', ensemble_player22_metrics)
print('\n')

VotingRegressor Metrics on Player 22: (0.4294109351393311, 0.9881350320107591)




The code is using the trained models (RandomForestRegressor, XGBoost, GradientBoostingRegressor, and VotingRegressor) to make predictions on the Player 22 data.

In [42]:
rf_predictions = rf_model.predict(X_test_player22_scaled)
xgb_predictions = xgb_model.predict(X_test_player22_scaled)
gb_predictions = gb_model.predict(X_test_player22_scaled)
ensemble_predictions = best_ensemble_model.predict(X_test_player22_scaled)

Calling the `evaluate_model` function for each model, passing in the respective model, the scaled features (`X_test_player22_scaled`), and the target variable (`y_test_player22`). The function returns the evaluation metrics for each model,
which are then printed out.

In [43]:

rf_metrics_test = evaluate_model(rf_model, X_test_player22_scaled, y_test_player22)
xgb_metrics_test = evaluate_model(xgb_model, X_test_player22_scaled, y_test_player22)
gb_metrics_test = evaluate_model(gb_model, X_test_player22_scaled, y_test_player22)
ensemble_metrics_test = evaluate_model(best_ensemble_model, X_test_player22_scaled, y_test_player22)

print('RandomForestRegressor Test Metrics (player 22):', rf_metrics_test)
print ("  ")
print('XGBoost Test Metrics(player 22):', xgb_metrics_test)
print ("  ")
print('GradientBoostingRegressor Test Metrics(player 22):', gb_metrics_test)
print ("  ")
print('VotingRegressor Test Metrics:(player 22)', ensemble_metrics_test)

RandomForestRegressor Test Metrics (player 22): (0.41478468215603703, 0.9888725087608506)
  
XGBoost Test Metrics(player 22): (0.44372538805466455, 0.9872474358809686)
  
GradientBoostingRegressor Test Metrics(player 22): (0.47413183465303144, 0.9847915517766906)
  
VotingRegressor Test Metrics:(player 22) (0.4294109351393311, 0.9881350320107591)


We ended up going for the random forest regressor model with grid search because it has a lower mean absolute score as compared to all the other models.

In [44]:
import joblib

joblib.dump(rf_model, 'FifaPredictionmodel.pkl')

['FifaPredictionmodel.pkl']

In [45]:

joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [46]:

joblib.dump(scaler, 'Scaler.pkl')

['Scaler.pkl']

In [47]:
import sklearn
print(sklearn.__version__)


1.2.2
